In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader
import math
import torch.nn.functional as F
from models import TransformerModel, TransformerModelGrok, TransformerModelChatGPT, TransformerModelDeepseek, TransformerModelGrokOptimal, TransformerModelClaudeOptimal, TransformerModelCopilotOptimal, TransformerModelDeepseekOptimal, SEQUENCE_LENGTH, N_FEATURES, HIDDEN_SIZE, NUM_LAYERS, NUM_HEADS

In [43]:
class ActivityDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

In [44]:
def prepare_sequences(data, sequence_length):
    sequences = []
    labels = []
    
    for i in range(0, len(data) - sequence_length, sequence_length//2):  # 50% overlap
        seq = data[i:i + sequence_length]
        if len(seq) == sequence_length:
            sequences.append(seq)
            labels.append(seq['ActivityLabel'].mode()[0])  # Most common label in sequence
    
    return np.array(sequences), np.array(labels)

In [ ]:
def train_model():
    # Load data
    df = pd.read_csv('data\\combined\\merged_users_corrected.csv')
    
    # Prepare features and labels
    features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']
    X = df[features]
    y = df['ActivityLabel']
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=features)
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Create sequences
    X_scaled['ActivityLabel'] = y_encoded
    sequences, labels = prepare_sequences(X_scaled, SEQUENCE_LENGTH)
    
    # Remove ActivityLabel column from sequences
    sequences = sequences[:, :, :-1].astype(np.float32)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        sequences, labels, test_size=0.2, random_state=42
    )
    
    # Create datasets and dataloaders
    train_dataset = ActivityDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    test_dataset = ActivityDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    # Initialize model
    num_classes = len(label_encoder.classes_)
    model = TransformerModelCopilotOptimal(
        input_size=N_FEATURES,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        num_heads=NUM_HEADS,
        num_classes=num_classes
    )
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    best_accuracy = 0
    best_model_state = None
    
    # Training loop
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        for sequences, labels in train_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        current_accuracy = 100. * correct / total
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss/len(train_loader):.4f}, Accuracy: {current_accuracy:.2f}%')
        
        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_model_state = model.state_dict()
            print(f'New best accuracy: {best_accuracy:.2f}%')
    
    print(f'Training completed. Best accuracy: {best_accuracy:.2f}%')
    torch.save({
        'model_state_dict': best_model_state,
        'scaler': scaler,
        'label_encoder': label_encoder,
        'best_accuracy': best_accuracy,
    }, 'results\\TransformerModelCopilotOptimal_best.pth')

In [46]:
train_model()

Epoch 1/100, Loss: 0.8019, Accuracy: 73.03%
Epoch 2/100, Loss: 0.3162, Accuracy: 91.36%
Epoch 3/100, Loss: 0.2139, Accuracy: 94.37%
Epoch 4/100, Loss: 0.1793, Accuracy: 95.43%
Epoch 5/100, Loss: 0.1640, Accuracy: 95.43%
Epoch 6/100, Loss: 0.1421, Accuracy: 96.50%
Epoch 7/100, Loss: 0.1382, Accuracy: 96.20%
Epoch 8/100, Loss: 0.1130, Accuracy: 96.75%
Epoch 9/100, Loss: 0.1184, Accuracy: 96.50%
Epoch 10/100, Loss: 0.0966, Accuracy: 97.32%
Epoch 11/100, Loss: 0.0909, Accuracy: 97.57%
Epoch 12/100, Loss: 0.1188, Accuracy: 96.36%
Epoch 13/100, Loss: 0.0916, Accuracy: 97.32%
Epoch 14/100, Loss: 0.0800, Accuracy: 97.81%
Epoch 15/100, Loss: 0.0730, Accuracy: 97.78%
Epoch 16/100, Loss: 0.0587, Accuracy: 98.33%
Epoch 17/100, Loss: 0.0622, Accuracy: 98.22%
Epoch 18/100, Loss: 0.0436, Accuracy: 98.66%
Epoch 19/100, Loss: 0.0708, Accuracy: 97.78%
Epoch 20/100, Loss: 0.1115, Accuracy: 96.85%
Epoch 21/100, Loss: 0.0622, Accuracy: 98.25%
Epoch 22/100, Loss: 0.0464, Accuracy: 98.61%
Epoch 23/100, Loss: